# Australia Domestic Flights

We use a database provided by the **Bureau of Infrastructure and Transport Research Economics** downloaded from https://data.gov.au/dataset/ds-dga-c5029f2a-39b3-4aef-8ae1-73e7962f6170/details

The dataset contain 11 instances:

+ City1: A city each aircraft takes-off from
+ City2: A city each aircraft lands on.
+ Month: (I couldn’t find any definitions)
+ Passenger_Trips: (I couldn’t find any definitions)
+ Aircraft_Trips: The number of flight stages. A return gflight counts as two aircraft trips.
+ Passenger_Load_Factor: The total revenue passenger kilometres performed as a percentage of the total available seat kilometres.
+ Distance_GC_(km): Great Circle Distances between 2 airports
+ RPKs: Revenue Passenger Kilometres. calculated by multiplying the number of revenue passengers travelling on each flight stage, by the distance in kilometres between the ports. the distances used are great circle kilometres.
+ ASKs: It stands for “Available Seat Kilometres”. It is calculates by multiplying the number of seats availble on each flight stage, by the distance in kilometres between the ports. The distance used are Great Circle Distances.
+ Seats: Seats available in each route in that month
+ “Flight Statge” means the operation of an aircraft from take-off to landing.

*We will answer the following questions:*

+ Most popular routes
+ Which city shows a greater growth in flights **land on**?
+ Which was the best year for the australian domestic flights?
+ New cities added in the routes.

*Potential questions.*

+ Cities that have more growth potential.
+ **Try any algorithm of time series to predict the development of the flights**


In [40]:
import scipy
from scipy import stats

#librerías necesarias:

# Data handling
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Visualization
import seaborn as sns
#import plotly.express as px
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation

%matplotlib inline 
pd.set_option("max_rows", None)

## Dataset

In [41]:
data = pd.read_csv('https://data.gov.au/data/dataset/c5029f2a-39b3-4aef-8ae1-73e7962f6170/resource/677d307f-6a1f-4de4-9b85-5e1aa7074423/download/dom_citypairs_web.csv')

In [42]:
data.head()

,City1,City2,Month,Passenger_Trips,Aircraft_Trips,Passenger_Load_Factor,Distance_GC_(km),RPKs,ASKs,Seats,Year,Month_num
0,ADELAIDE,ALICE SPRINGS,30682,15743,143,81.8,1316,20717788.0,25327369.0,19246,1984,1
1,ADELAIDE,BRISBANE,30682,3781,32,89.8,1622,6132782.0,6829379.0,4210,1984,1
2,ADELAIDE,CANBERRA,30682,1339,12,94.7,972,1301508.0,1374348.0,1414,1984,1
3,ADELAIDE,DARWIN,30682,3050,33,66.8,2619,7987950.0,11958009.0,4566,1984,1
4,ADELAIDE,GOLD COAST,30682,1596,16,88.5,1607,2564772.0,2898047.0,1803,1984,1


In [43]:
data.shape

(24158, 12)

In [44]:
data.dtypes

City1                     object
City2                     object
Month                      int64
Passenger_Trips            int64
Aircraft_Trips             int64
Passenger_Load_Factor     object
Distance_GC_(km)           int64
RPKs                     float64
ASKs                     float64
Seats                      int64
Year                       int64
Month_num                  int64
dtype: object

### Period Stamp

In [45]:
data['Date'] = data['Month_num'].map(str) + '/' + data['Year'].map(str) 

In [46]:
data.head()

,City1,City2,Month,Passenger_Trips,Aircraft_Trips,Passenger_Load_Factor,Distance_GC_(km),RPKs,ASKs,Seats,Year,Month_num,Date
0,ADELAIDE,ALICE SPRINGS,30682,15743,143,81.8,1316,20717788.0,25327369.0,19246,1984,1,1/1984
1,ADELAIDE,BRISBANE,30682,3781,32,89.8,1622,6132782.0,6829379.0,4210,1984,1,1/1984
2,ADELAIDE,CANBERRA,30682,1339,12,94.7,972,1301508.0,1374348.0,1414,1984,1,1/1984
3,ADELAIDE,DARWIN,30682,3050,33,66.8,2619,7987950.0,11958009.0,4566,1984,1,1/1984
4,ADELAIDE,GOLD COAST,30682,1596,16,88.5,1607,2564772.0,2898047.0,1803,1984,1,1/1984


In [47]:
data_1 = pd.to_datetime(data['Date'])
data['Date'] = pd.PeriodIndex(data_1, freq='M')
data[:5]

,City1,City2,Month,Passenger_Trips,Aircraft_Trips,Passenger_Load_Factor,Distance_GC_(km),RPKs,ASKs,Seats,Year,Month_num,Date
0,ADELAIDE,ALICE SPRINGS,30682,15743,143,81.8,1316,20717788.0,25327369.0,19246,1984,1,1984-01
1,ADELAIDE,BRISBANE,30682,3781,32,89.8,1622,6132782.0,6829379.0,4210,1984,1,1984-01
2,ADELAIDE,CANBERRA,30682,1339,12,94.7,972,1301508.0,1374348.0,1414,1984,1,1984-01
3,ADELAIDE,DARWIN,30682,3050,33,66.8,2619,7987950.0,11958009.0,4566,1984,1,1984-01
4,ADELAIDE,GOLD COAST,30682,1596,16,88.5,1607,2564772.0,2898047.0,1803,1984,1,1984-01


## Most Popular Routes

In this section we are going to explore the most popular domestic routes and how they change during the years.

### Routes along the years

Seats available in each route during the years,

In [48]:
routes = pd.DataFrame(data, columns=['City1', 'City2', 'Seats', 'Date'])

In [50]:
routes['Routes'] = routes['City1'] + '-' + routes['City2'] 
routes